In [18]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 15)

LARGE_FIGSIZE = (12, 8)

In [3]:
df = pd.read_csv('C:/Users/Kate/PycharmProjects/test/test/P9-ConsumerComplaints.csv',
                 header = 0,
                 na_values=['N/A'],
                 index_col = 'Complaint ID',
                 true_values=['Yes'],
                 false_values=['No'],
                 keep_default_na=False
                 )
df.head()

C:\Users\Kate\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed
Complaint ID,,,,,,,,,,,,,,,,,
468882,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,,,,Wells Fargo & Company,VA,24540,,NaN,Phone,2013-07-30,Closed with explanation,True,No
468889,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,,,,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,True,No
468879,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",,,,Santander Bank US,NY,10065,,NaN,Fax,2013-07-31,Closed,True,No
468949,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,,,,Wells Fargo & Company,GA,30084,,NaN,Web,2013-07-30,Closed with explanation,True,No
475823,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",,,,Franklin Credit Management,CT,6106,,NaN,Web,2013-07-30,Closed with explanation,True,No


In [10]:
df['Date Received'] = pd.to_datetime(df['Date Received'])

In [ ]:
# df['Consumer Disputed'] = df['Consumer Disputed'].applymap(lambda value: pd.to_numeric(value, errors='coerce'))

In [85]:
df.dtypes

Date Received                   datetime64[ns]
Product Name                            object
Sub Product                             object
Issue                                   object
Sub Issue                               object
                                     ...      
Submitted via                           object
Date Sent to Company                    object
Company Response to Consumer            object
Timely Response                           bool
Consumer Disputed                       object
Length: 17, dtype: object

In [7]:
date_start = pd.to_datetime('2013-07-29')
date_end = pd.to_datetime('2013-07-30')

In [12]:
# all product names
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end)]['Product Name'].unique()

array(['Consumer Loan', 'Bank account or service', 'Mortgage',
       'Debt collection', 'Credit card', 'Credit reporting',
       'Student loan'], dtype=object)

In [13]:
# complaints for each product
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end)][['Product Name','Issue']].sort_values('Product Name')

,Product Name,Issue
Complaint ID,,
469537,Bank account or service,Problems caused by my funds being low
470611,Bank account or service,"Account opening, closing, or management"
479508,Bank account or service,Problems caused by my funds being low
469446,Bank account or service,Deposits and withdrawals
470633,Bank account or service,"Account opening, closing, or management"
...,...,...
469340,Student loan,Repaying your loan
467427,Student loan,Repaying your loan
479648,Student loan,Repaying your loan


In [14]:
# how much with timely response
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end) & (df['Timely Response'] == True)][['Product Name','Timely Response']].groupby('Product Name').sum()

,Timely Response
Product Name,
Bank account or service,34
Consumer Loan,10
Credit card,26
Credit reporting,22
Debt collection,25
Mortgage,75
Student loan,7


In [16]:
# how much disputed by customer
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end) & (df['Consumer Disputed'] == True)][['Product Name','Consumer Disputed']].groupby('Product Name').sum()

,Consumer Disputed
Product Name,


In [33]:
# how much with timely response sorted
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end)].groupby('Product Name').sum().sort_values('Timely Response', ascending=False)

,Timely Response
Product Name,
Mortgage,75
Bank account or service,34
Credit card,26
Debt collection,25
Credit reporting,22
Consumer Loan,10
Student loan,7
